In [1]:
import os
import re
import pandas as pd
import numpy as np

In [2]:
import PyPDF2
import tabula
from tabula import read_pdf

import textract

In [3]:
cd

/Users/andrew.vangilder


In [4]:
os.chdir('Desktop/local_data')

OPTIONS: Tabula, PyPDF2, textract, manual copy-paste then process

### Errors with Tabula python library

In [5]:
#pdf = read_pdf('data/site-histology.pdf','rb')

In [6]:
#df = tabula.read_pdf("data/site-histology.pdf", encoding='utf-8', spreadsheet=True, pages='1-6041')

In [8]:
#type(pdf)

### Textract

In [9]:
text = textract.process('site-histology2.pdf', method='pdfminer')

In [13]:
text[:1000]

'ICD-0-3 SEER SITE/HISTOLOGY VALIDATION LIST     March 23, 2018                                                                                          1\n\nMarch 23, 2018\n\nICD-0-3 SEER SITE/HISTOLOGY VALIDATION LIST\n\nThis file is intended as a reference file for ICD-O-3 only and is not to be used for casefinding purposes. The ICD-O-3 site/type\nvalidation program was modified to allow only for the site/histology/behavior combinations listed in this publication. All other cases\nmust be reviewed.\nSubmit any comments or questions about this document to:\nAsk A SEER Registrar\nhttps://seer.cancer.gov/registrars/contact.html\n\nor\n\nseerweb@imsweb.com\n\n\x0cICD-0-3 SEER SITE/HISTOLOGY VALIDATION LIST     March 23, 2018                                                                                          2\n\nLIP C000-C006,C008-C009\n\nNEOPLASM\n\nCARCINOMA, NOS\n\nCARCINOMA, UNDIFF., NOS\n\nGIANT & SPINDLE CELL CARCINOMA\n\nPAPILLARY CARCINOMA, NOS\n\nSQUAMOUS CELL CARCINOMA, N

In [22]:
for m in re.finditer(r'(?<=(\n\n)([A-Z]))+', text[:1000]):
    print('%02d-%02d: %s ... %s' % (m.start(), m.end(), m.group(0),text[m.start()-5:m.end()+10]))

156-156:  ...  1

March 23, 2
172-172:  ... 18

ICD-0-3 SEE
217-217:  ... ST

This file i
806-806:  ...  2

LIP C000-C0
831-831:  ... 09

NEOPLASM

C
841-841:  ... SM

CARCINOMA, 
857-857:  ... OS

CARCINOMA, 
882-882:  ... OS

GIANT & SPI
914-914:  ... MA

PAPILLARY C
940-940:  ... OS

SQUAMOUS CE


In [14]:
for m in re.finditer(r'([A-Z]{0})([0-9]{4})/([0-9]{1})([A-z]?)', text[:1000]):
    print('%02d-%02d: %s ... %s' % (m.start(), m.end(), m.group(0),m))

### PyPDF2 

In [26]:
pdfFileObj = open('site-histology2.pdf', 'rb')

In [27]:
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

In [28]:
pdfReader.numPages

380

From pyPDF2 documentation:

extractText()

Locate all text drawing commands, in the order they are provided in the content stream, and extract the text. This works well for some PDF files, but poorly for others, depending on the generator used. This will be refined in the future. Do not rely on the order of text coming out of this function, as it will change if this function is made more sophisticated.

Returns:	a unicode string object.

In [29]:
def get_page(page):
    pageObj = pdfReader.getPage(page)
    return pageObj.extractText()

In [30]:
def get_contents(page):
    pageObj = pdfReader.getPage(page)
    return pageObj.getContents().getData()

In [33]:
text = get_page(5)

In [34]:
text

u'ICD-0-3 SEER SITE/HISTOLOGY VALIDATION LIST     March 23, 2018                                                                                          6BASEOFTONGUE C019LYMPHOEPITHELIAL CARCINOMA8088081/2Bowen disease8082/3Lymphoepithelial carcinoma8083/3Basaloid squamous cell carcinoma8084/3Squamous cell carcinoma, clear cell type8085/3Squamous cell carcinoma, HPV-positive8086/3Squamous cell carcinoma, HPV-negativeADENOCARCINOMA, NOS8148140/2Adenocarcinoma in situ8140/3Adenocarcinoma, NOS8141/3Scirrhous adenocarcinoma8143/3Superficial spreading adenocarcinoma8147/3Basal cell adenocarcinomaADENOID CYSTIC & CRIBRIFORM CA.8208200/3Adenoid cystic carcinoma8201/2Cribriform carcinoma in situ8201/3Cribriform carcinomaBRONCHIOLO-ALVEOLAR ADENOCA.8258255/3Adenocarcinoma with mixed subtypesMUCOEPIDERMOID CARCINOMA8438430/3Mucoepidermoid carcinomaMUCINOUS ADENOCARCINOMA8488480/3Mucinous adenocarcinoma8481/3Mucin-producing adenocarcinomaADENOSQUAMOUS CARCINOMA8568560/3Adenosquamous carcinoma85

In [ ]:
contents = get_contents(2)

In [ ]:
contents.decode("utf-8")[:1000]

In [ ]:
target_df = pd.DataFrame(data=None, columns=['Site Recode','Site Description','Histo','Histology Description','Histology/Behavior','Histology/Behavior Description'])

In [ ]:
target_df

for page in range(pdfReader.numPages):
    look for site recode

## histology/behavior
### status: very good


In [ ]:
for m in re.finditer(r'([A-Z]{0})([0-9]{4})/([0-9]{1})([A-z]?)', text):
    print('%02d-%02d: %s' % (m.start(), m.end(), m.group(0)[:-1]))

## Site Recode:
### status:

In [ ]:
#([0-9])(?=[A-Z])
#([A-Z])([0-9]).+(?=[0-9][A-Z])
#([A-Z])([0-9]).+(?=[0-9][A-Z])?
#(C[0-9]{3})
#(?<=:).+?(?=/)
for m in re.finditer(r'(?<=(C([0-9]{3}))).{9}', text):
    print('%02d-%02d: %s' % (m.start(), m.end(), m.group(0)))

## Histology description:
### Status: Okay

In [ ]:
rand_page = int(np.random.uniform(380))
print(rand_page)

In [ ]:
text = get_page(rand_page)
for m in re.finditer(r'([A-Z,)])+?(?=( C([0-9]{3})))', text):
    print('%02d-%02d: %s page %d' % (m.start(), m.end(), m.group(0), rand_page))

In [ ]:
text

In [ ]:
for m in re.finditer(r'([0-9]{4})/([0-9]{1})', text):
    pd
    print('%02d-%02d: %s' % (m.start(), m.end(), m.group(0)))

4 int + / + int

In [ ]:
#re = \d{4}'\'\d{1}
x = page

In [ ]:
page = get_page(1)

In [ ]:
get_page(5).split(' ')
#.split('/')

In [ ]:
result_string = ''
re.match("d",page)

In [ ]:
type(get_page(1))

In [ ]:
print('this page contains %d characters' % (len(get_page(2))))

In [ ]:
regex = re.compile(r'([0-9]{2}/)+([0-9]{1})([A-Z])')
regex2 = re.compile(r'/([0-9]{1})')
regex3 = re.compile(r'([0-9]{2}/)([0-9]{1})([A-Z])')

page = regex3.split(get_page(2))
page2 = regex2.split(get_page(1))

In [ ]:
for line in page:
    print(line)

In [ ]:
for line in page2:
    print(line)

In [ ]:
len(page)

In [ ]:
print(1)